# Displaying platform and device information with PyOpenCL

Elwin van 't Wout

PUC Chile

25-9-2024

This tutorial shows how to retrieve the information of the platforms and devices that are available to OpenCL.

To use a GPU, select the GPU runtime environment in Google Colab (in the menu, click Runtime -> Change Runtime Type and select the GPU). We also need to configure the virtual machine, install the hardware drivers, and load the PyOpenCL library. This may take a few minutes.

In [1]:
!sudo apt update
!sudo apt install -y nvidia-cuda-toolkit pocl-opencl-icd

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,001 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,113 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu

In [2]:
!pip install pyopencl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.1/698.1 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 9.5 MB/s eta 0:00:00


In [3]:
import pyopencl as cl

/usr/local/lib/python3.10/dist-packages/pytools/persistent_dict.py:63: RecommendedHashNotFoundWarning: Unable to import recommended hash 'siphash24.siphash13', falling back to 'hashlib.sha256'. Run 'python3 -m pip install siphash24' to install the recommended hash.
  warn("Unable to import recommended hash 'siphash24.siphash13', "


OpenCL uses 'platforms' as the basic environment to store information on the implementation platform. The function ```get_platforms()``` returns a list with the different platforms available. Each platform object contains information such as name and vendor.

In [4]:
print('OpenCL Platforms')
for count, platform in enumerate(cl.get_platforms()):
    print('')
    print('Platform ' + str(count) + ' - Name:    ' + platform.name)
    print('Platform ' + str(count) + ' - Vendor:  ' + platform.vendor)
    print('Platform ' + str(count) + ' - Version: ' + platform.version)
    print('Platform ' + str(count) + ' - Profile: ' + platform.profile)

OpenCL Platforms

Platform 0 - Name:    NVIDIA CUDA
Platform 0 - Vendor:  NVIDIA Corporation
Platform 0 - Version: OpenCL 3.0 CUDA 12.2.138
Platform 0 - Profile: FULL_PROFILE

Platform 1 - Name:    Portable Computing Language
Platform 1 - Vendor:  The pocl project
Platform 1 - Version: OpenCL 2.0 pocl 1.8  Linux, None+Asserts, RELOC, LLVM 11.1.0, SLEEF, DISTRO, POCL_DEBUG
Platform 1 - Profile: FULL_PROFILE


Each platform object has a device, or multiple devices. These are the actual compute devices that are available for calculations.

In [5]:
print('OpenCL Devices')
for p, platform in enumerate(cl.get_platforms()):
    print('')
    print('Platform ' + str(p) + ' - Name:  ' + platform.name)
    for d, device in enumerate(platform.get_devices()):
        print('')
        print('Device ' + str(p) + '.' + str(d) + ' - Name:  ' + device.name)
        print('Device ' + str(p) + '.' + str(d) + ' - Type:  ' + cl.device_type.to_string(device.type))
        print('Device ' + str(p) + '.' + str(d) + ' - Max Clock Speed:  {0} Mhz'.format(device.max_clock_frequency))
        print('Device ' + str(p) + '.' + str(d) + ' - Compute Units:  {0}'.format(device.max_compute_units))
        print('Device ' + str(p) + '.' + str(d) + ' - Local Memory:  {0:.0f} KB'.format(device.local_mem_size/1024.0))
        print('Device ' + str(p) + '.' + str(d) + ' - Constant Memory:  {0:.0f} KB'.format(device.max_constant_buffer_size/1024.0))
        print('Device ' + str(p) + '.' + str(d) + ' - Global Memory: {0:.0f} GB'.format(device.global_mem_size/1073741824.0))
        print('Device ' + str(p) + '.' + str(d) + ' - Max Work Group Size: {0:.0f}'.format(device.max_work_group_size))

OpenCL Devices

Platform 0 - Name:  NVIDIA CUDA

Device 0.0 - Name:  Tesla T4
Device 0.0 - Type:  ALL | GPU
Device 0.0 - Max Clock Speed:  1590 Mhz
Device 0.0 - Compute Units:  40
Device 0.0 - Local Memory:  48 KB
Device 0.0 - Constant Memory:  64 KB
Device 0.0 - Global Memory: 15 GB
Device 0.0 - Max Work Group Size: 1024

Platform 1 - Name:  Portable Computing Language

Device 1.0 - Name:  pthread-Intel(R) Xeon(R) CPU @ 2.20GHz
Device 1.0 - Type:  ALL | CPU
Device 1.0 - Max Clock Speed:  2199 Mhz
Device 1.0 - Compute Units:  2
Device 1.0 - Local Memory:  512 KB
Device 1.0 - Constant Memory:  512 KB
Device 1.0 - Global Memory: 11 GB
Device 1.0 - Max Work Group Size: 4096


The statistics provide information about the type of device and the hardware characteristics like number of compute units and memory size.

To perform instructions on the compute devices, a `Context` needs to be created. If multiple devices are available, one has to select the device.

The easiest way to create a `Context` is to let OpenCL decide how to configure the object and which device to select.

In [6]:
some_ctx = cl.create_some_context()
some_device = some_ctx.devices[0]
print("Device name:", some_device.name)
print("Device type:", cl.device_type.to_string(some_device.type))

Device name: Tesla T4
Device type: ALL | GPU


One can also specify which device to use explicitly. From the above output, we already know that the first platform considers CUDA and the second POCL.

In [7]:
platforms_cuda = cl.get_platforms()[0]
platforms_pocl = cl.get_platforms()[1]

devices_gpu = platforms_cuda.get_devices(device_type=cl.device_type.GPU)
devices_cpu = platforms_pocl.get_devices(device_type=cl.device_type.CPU)

ctx_gpu = cl.Context(devices=devices_gpu)
ctx_cpu = cl.Context(devices=devices_cpu)

print("GPU context includes:")
for device in devices_gpu:
  print(" Device name:", device.name)
  print(" Device type:", cl.device_type.to_string(device.type))

print("CPU context includes:")
for device in devices_cpu:
  print(" Device name:", device.name)
  print(" Device type:", cl.device_type.to_string(device.type))

GPU context includes:
 Device name: Tesla T4
 Device type: ALL | GPU
CPU context includes:
 Device name: pthread-Intel(R) Xeon(R) CPU @ 2.20GHz
 Device type: ALL | CPU


Notice that the context can can have multiple devices, for example when multiple GPUs are available.